# Section 1: Get Started 

In [14]:
from dotenv import load_dotenv
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY', 'YourAPIKey')

In [6]:
llm = OpenAI(temperature=1)
llm.predict("hi!")

'\\\\\n\nHello!'

In [7]:
chat_model = ChatOpenAI()
chat_model.predict("hi!")

'Hello! How can I assist you today?'

In [26]:
text = "What would be a good company name for a company that makes colorful socks?"
response1 = llm.predict(text)
response2 = chat_model.predict(text)
print(response1)
print(response2)
# print(type(response1)) # <class 'str'>
# print(type(response2)) # <class 'str'>
# as you can tell here its string in, string out



Rainbow Socks Co.
Colorful Soles
<class 'str'>
<class 'str'>


In [15]:
messages = [HumanMessage(content=text)]

cr1 = llm.predict_messages(messages)
print(cr1)
cr2 = chat_model.predict_messages(messages)
print(cr2)
# here it is message in, message out - regardless of llm or chat model 

content="\n\nSockin' It Up!" additional_kwargs={} example=False
content='SockSplash' additional_kwargs={} example=False



The standard interface that LangChain provides has two methods:

predict: Takes in a string, returns a string
predict_messages: Takes in a list of messages, returns a message.

For both these methods, you can also pass in parameters as keyword arguments. For example, you could pass in temperature=0 to adjust the temperature that is used from what the object was configured with. Whatever values are passed in during run time will always override what the object was configured with.

In [25]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
# print(type(chat_prompt)) # <class 'langchain.prompts.chat.ChatPromptTemplate'>

cprompt = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")
# print(type(cprompt)) # <class 'list'>
cr3 = chat_model.predict_messages(cprompt)
cr4 = llm.predict_messages(cprompt)
print(cr3)
print(cr4)
# print(type(cr3)) # <class 'langchain.schema.messages.AIMessage'>
# print(type(cr4)) # <class 'langchain.schema.messages.AIMessage'>

# in the case of just simple prompt templates, you can just think about it like F Strings;

content="J'adore la programmation." additional_kwargs={} example=False
content="\n\nSystem: J'adore la programmation." additional_kwargs={} example=False
<class 'langchain.schema.messages.AIMessage'>


### Output parsers
OutputParsers convert the raw output of an LLM into a format that can be used downstream. There are few main types of OutputParsers, including:

- Convert text from LLM into structured information (e.g. JSON)
- Convert a ChatMessage into just a string
- Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.
- For full information on this, see the section on output parsers.

In this getting started guide, we will write our own output parser - one that converts a comma separated list into a list.

In [29]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("ayy, lmao - wtf??")


['ayy', 'lmao - wtf??']

# PromptTemplate + LLM + OutputParser
We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser. This is a convenient way to bundle up a modular piece of logic. Let's see it in action!



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']

Note that we are using the | syntax to join these components together. This | syntax is called the LangChain Expression Language. To learn more about this syntax, read the documentation here.